# Neo4j와 LangChain을 활용한 영화 추천 시스템

---

## 1. Neo4J AuraDB 환경 설정

In [1]:
import os
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()

True

In [2]:
from langchain_neo4j import Neo4jGraph

# LangChain 도구 활용 - DB 연결 객체 초기화 
graph = Neo4jGraph( 
    url=os.getenv("NEO4J_URI"), 
    username=os.getenv("NEO4J_USERNAME"), 
    password=os.getenv("NEO4J_PASSWORD"),
)

In [15]:
# 테스트 쿼리 실행 
cypher_query = """
MATCH (n:Movie)
RETURN COUNT(n) AS Movie_Count
"""

graph.query(cypher_query)

[{'Movie_Count': 4803}]

---

## 2. **Movie 노드** : 임베딩 필드 추가 및 벡터 인덱스 생성

- **Movie 노드**에 텍스트 데이터를 벡터화한 **임베딩 필드**를 추가함
- 영화 제목, 줄거리 등의 텍스트 정보를 **고차원 벡터**로 변환하여 저장함
- 벡터화된 데이터를 효율적으로 검색하기 위한 **벡터 인덱스**를 생성함
- 인덱스 생성 시 **벡터 차원**, **유사도 계산 방식**, **거리 함수** 등을 지정함
- 임베딩과 벡터 인덱스를 통해 **의미적 검색**의 기반을 구축함

### 2.1 임베딩 모델 초기화

- **OpenAI 임베딩 모델**을 활용하여 텍스트를 벡터로 변환하는 환경을 설정함
- 임베딩 모델은 텍스트의 **의미적 특성**을 수치화된 벡터로 표현함

In [3]:
from langchain_openai import OpenAIEmbeddings

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-3-small") # 1536차원 임베딩

### 2.2 벡터 인덱스 생성

- Neo4j에서 **벡터 인덱스**를 생성하여 영화 줄거리의 임베딩 벡터를 효율적으로 검색할 수 있게 함
- **movie_content_embeddings**라는 이름의 벡터 인덱스를 Movie 노드의 **content_embedding** 필드에 적용함 (필드를 새로 추가)
- 벡터 차원을 **1536차원**으로 설정하여 OpenAI의 text-embedding-3-small 모델과 호환되도록 함
- 유사도 계산 방식으로 **코사인 유사도**를 선택하여 벡터 간 각도 기반의 의미적 유사성을 측정함
- 쿼리에 **IF NOT EXISTS** 조건을 포함하여 중복 생성을 방지함

In [17]:
# 벡터 인덱스 생성
create_vector_index_query = """
// 영화 콘텐츠 임베딩을 위한 벡터 인덱스 생성
// IF NOT EXISTS: 이미 존재하는 경우 중복 생성 방지
CREATE VECTOR INDEX movie_content_embeddings IF NOT EXISTS 

// Movie 노드의 content_embedding 속성에 인덱스 적용
FOR (m:Movie) ON m.content_embedding 

// 벡터 인덱스 설정 옵션 (OPTIONS 절은 인덱스 생성 쿼리의 가장 마지막에 위치해야 함)
OPTIONS {
  indexConfig: {
    `vector.dimensions`: 1536,
    `vector.similarity_function`: 'cosine'
  }
}
"""
graph.query(create_vector_index_query)

[]

In [4]:
# 벡터 인덱스 확인
check_vector_index_query = """
SHOW VECTOR INDEXES
"""
vector_indexes = graph.query(check_vector_index_query)
for index in vector_indexes:
    # 벡터 인덱스 정보 출력
    print(f"Index Name: {index['name']}")
    print(f"Type: {index['type']}")    
    print(f"Property Key: {index['properties']}")
    print("-" * 40)

Index Name: movie_content_embeddings
Type: VECTOR
Property Key: ['content_embedding']
----------------------------------------


### 2.3 임베딩 생성 및 저장

- 각 영화 제목, 태그라인, 개요에 대해 **OpenAI 임베딩**을 생성하는 과정 수행
- 빈 문자열인 경우 처리를 **건너뛰는** 예외 처리 포함
- 생성된 임베딩을 `db.create.setNodeVectorProperty` 프로시저를 통해 **content_embedding** 속성으로 저장
- 단순하게 노드 속성에 임베딩값을 추가하지 않고, db.create.setNodeVectorProperty 프로시저를 이용하여 속성을 추가하는 이유는 임베딩 벡터를 벡터 전용 구조로 저장하여 Neo4j의 그래프 데이터 과학(GDS) 라이브러리와 연동 가능한 ANN(Approximate Nearest Neighbor) 기반 유사도 검색을 효율적으로 수행하기 위해서입니다. 단순 속성으로 저장하면 검색 시 모든 노드를 순회하며 계산해야 하므로 성능이 매우 떨어지지만, 전용 프로시저를 사용하면 대규모 데이터에서도 빠른 벡터 검색과 최적화된 유사도 계산이 가능해집니다.

In [5]:
# 영화 제목과 줄거리 가져오기
movies_query = """
MATCH (m:Movie)
WHERE m.title IS NOT NULL
RETURN m.id AS id, m.title AS title, m.overview AS overview, m.tagline AS tagline
"""
movies = graph.query(movies_query)

# 배치 크기 설정
BATCH_SIZE = 100

# 임베딩 생성 및 저장 (배치 처리)
for i in range(0, len(movies), BATCH_SIZE):
    batch = movies[i:i+BATCH_SIZE]
    batch_texts = []
    batch_ids = []
    
    # 배치 데이터 준비
    for movie in batch:
        # overview와 tagline을 "\n\n"으로 결합
        content_text = f"{movie['title']}"
        if movie['tagline']:
            content_text += f"\n\n{movie['tagline']}"
        if movie['overview']:
            content_text += f"\n\n{movie['overview']}"
        
        if content_text.strip():  # 빈 문자열 확인
            batch_texts.append(content_text)
            batch_ids.append(movie['id'])
    
    try:
        if batch_texts:
            # 배치 단위로 OpenAI 임베딩 생성
            batch_embeddings = embeddings.embed_documents(batch_texts) # 100개 동시에 임베딩 생성
            
            # UNWIND를 사용한 배치 업데이트
            """
            batch_data = [
                {'id': 101, 'embedding': [0.1, 0.2]},
                {'id': 102, 'embedding': [0.3, 0.4]},
                {'id': 103, 'embedding': [0.5, 0.6]}
            ]
            """
            batch_data = [{"id": article_id, "embedding": embedding_vector} 
                         for article_id, embedding_vector in zip(batch_ids, batch_embeddings)]
            
            batch_update_query = """
            // UNWIND를 사용하여 배치 데이터를 개별 행으로 변환
            UNWIND $batch AS item

            // 영화 ID로 해당 Movie 노드 찾기
            MATCH (m:Movie {id: item.id})

            // db.create.setNodeVectorProperty 프로시저를 호출하여 벡터 속성 설정
            // 첫 번째 인자: 대상 노드, 두 번째 인자: 속성 이름, 세 번째 인자: 벡터 값
            CALL db.create.setNodeVectorProperty(m, 'content_embedding', item.embedding)

            // 업데이트된 노드 수 반환
            RETURN count(m) as updated
            """
            
            result = graph.query(batch_update_query, params={"batch": batch_data})
            print(f"배치 처리 완료: {i+1}~{min(i+len(batch_texts), len(movies))} / {len(movies)}, 업데이트됨: {result[0]['updated']}")
            
    except Exception as e:
        print(f"배치 임베딩 생성 실패 (배치 인덱스 {i}): {str(e)}")

print(f"영화 임베딩 업데이트 완료!! 총 {len(movies)}개 처리")

배치 처리 완료: 1~100 / 4803, 업데이트됨: 100
배치 처리 완료: 101~200 / 4803, 업데이트됨: 100
배치 처리 완료: 201~300 / 4803, 업데이트됨: 100
배치 처리 완료: 301~400 / 4803, 업데이트됨: 100
배치 처리 완료: 401~500 / 4803, 업데이트됨: 100
배치 처리 완료: 501~600 / 4803, 업데이트됨: 100
배치 처리 완료: 601~700 / 4803, 업데이트됨: 100
배치 처리 완료: 701~800 / 4803, 업데이트됨: 100
배치 처리 완료: 801~900 / 4803, 업데이트됨: 100
배치 처리 완료: 901~1000 / 4803, 업데이트됨: 100
배치 처리 완료: 1001~1100 / 4803, 업데이트됨: 100
배치 처리 완료: 1101~1200 / 4803, 업데이트됨: 100
배치 처리 완료: 1201~1300 / 4803, 업데이트됨: 100
배치 처리 완료: 1301~1400 / 4803, 업데이트됨: 100
배치 처리 완료: 1401~1500 / 4803, 업데이트됨: 100
배치 처리 완료: 1501~1600 / 4803, 업데이트됨: 100
배치 처리 완료: 1601~1700 / 4803, 업데이트됨: 100
배치 처리 완료: 1701~1800 / 4803, 업데이트됨: 100
배치 처리 완료: 1801~1900 / 4803, 업데이트됨: 100
배치 처리 완료: 1901~2000 / 4803, 업데이트됨: 100
배치 처리 완료: 2001~2100 / 4803, 업데이트됨: 100
배치 처리 완료: 2101~2200 / 4803, 업데이트됨: 100
배치 처리 완료: 2201~2300 / 4803, 업데이트됨: 100
배치 처리 완료: 2301~2400 / 4803, 업데이트됨: 100
배치 처리 완료: 2401~2500 / 4803, 업데이트됨: 100
배치 처리 완료: 2501~2600 / 4803, 업데이트됨: 100
배치 처리 완

---

## 3. **의미 기반 검색(Semantic Search)** 구현

### 3.1 기본 검색

- `semantic_movie_search` 함수는 **텍스트 기반 의미 검색**을 구현함
- 입력된 검색어를 **OpenAI 임베딩**으로 변환하는 과정 포함
- Neo4j의 **벡터 인덱스 검색** 기능(`db.index.vector.queryNodes`)을 활용
- 검색 결과로 영화 **제목**, **개봉일**, **평점**과 **유사도 점수**를 반환

In [6]:
def semantic_movie_search(search_text, top_k=5):
    """
    텍스트 쿼리를 받아 의미적으로 가장 유사한 영화를 반환합니다.
    
    매개변수:
        search_text (str): 검색할 텍스트 쿼리
        top_k (int): 반환할 최대 결과 수 (기본값: 5)
        
    반환값:
        list: 유사도 점수가 높은 순으로 정렬된 영화 정보 목록
    """
    # 검색 텍스트의 임베딩 생성 (OpenAI API를 사용하여 텍스트를 벡터로 변환)
    query_embedding = embeddings.embed_query(search_text)
    
    # Neo4j 벡터 검색 쿼리 실행
    # db.index.vector.queryNodes: Neo4j의 벡터 인덱스를 사용하여 유사한 노드를 검색하는 함수
    vector_search_query = """
    CALL db.index.vector.queryNodes(
        'movie_content_embeddings',      // 사용할 벡터 인덱스 이름
        $top_k,                          // 반환할 결과 수
        $query_embedding                 // 검색 쿼리의 임베딩 벡터
    ) YIELD node, score                  // 검색 결과 노드와 유사도 점수 반환
    RETURN node.title AS title,          // 영화 제목
           node.released AS released,    // 개봉일
           node.rating AS rating,        // 평점
           score AS similarity           // 유사도 점수
    ORDER BY similarity DESC             // 유사도 점수 기준 내림차순 정렬
    """
    
    # Neo4j 그래프 데이터베이스에 쿼리 실행
    # top_k와 query_embedding을 파라미터로 전달
    results = graph.query(
        vector_search_query,
        params={"top_k": top_k, "query_embedding": query_embedding}
    )
    
    # 검색 결과 반환 (영화 제목, 개봉일, 평점, 유사도 점수 포함)
    return results

In [7]:
# 임베딩 기반 의미 검색 (영어 예시)
search_text = "a Drama movie about artificial intelligence and reality"

# 의미 검색 실행
results = semantic_movie_search(search_text)

for result in results:
    print(f"{result['title']} - 유사도: {result['similarity']:.4f}, " +
          f"평점: {result['rating']}, 개봉: {result['released']}")

A.I. Artificial Intelligence - 유사도: 0.7671, 평점: 6.8, 개봉: 2001-06-29
Ex Machina - 유사도: 0.7633, 평점: 7.6, 개봉: 2015-01-21
Transcendence - 유사도: 0.7580, 평점: 5.9, 개봉: 2014-04-16
I, Robot - 유사도: 0.7479, 평점: 6.7, 개봉: 2004-07-15
Real Steel - 유사도: 0.7457, 평점: 6.6, 개봉: 2011-09-28


In [8]:
# 임베딩 기반 의미 검색 (한국어 예시)
search_text_kr = "인공지능과 현실에 관한 SF 영화"

# 의미 검색 실행
results_kr = semantic_movie_search(search_text_kr)

for result in results_kr:
    print(f"{result['title']} - 유사도: {result['similarity']:.4f}, " +
          f"평점: {result['rating']}, 개봉: {result['released']}")

Stealth - 유사도: 0.7107, 평점: 4.9, 개봉: 2005-07-28
The Matrix - 유사도: 0.7105, 평점: 7.9, 개봉: 1999-03-30
Real Steel - 유사도: 0.7091, 평점: 6.6, 개봉: 2011-09-28
A.I. Artificial Intelligence - 유사도: 0.7081, 평점: 6.8, 개봉: 2001-06-29
I, Robot - 유사도: 0.7068, 평점: 6.7, 개봉: 2004-07-15


### 3.2 하이브리드 검색

- **벡터 검색 + 키워드 필터링**:

    - `hybrid_movie_search` 함수는 **벡터 검색**과 **키워드 필터링(밑의 예제에서는 제외한다는게 아니라 포함한다는 의미)**을 결합한 검색 기능 구현
    - Neo4j의 **벡터 인덱스**를 활용하여 의미적 유사성 기반 검색 수행
    - **장르**와 **최소 평점** 등 추가 필터를 적용하여 검색 결과 정제

In [9]:
def hybrid_movie_search(search_text, genre=None, min_rating=None, top_k=5):
    """
    벡터 검색 및 키워드 필터링을 결합한 하이브리드 검색
    
    Args:
        search_text: 검색할 텍스트 쿼리
        genre: 필터링할 영화 장르 (선택적) 
        min_rating: 최소 평점 기준 (선택적)
        top_k: 반환할 최대 결과 수
        
    Returns:
        필터링된 영화 검색 결과 목록
    """
    # 검색 텍스트의 임베딩 생성 - OpenAI 임베딩 모델을 사용하여 텍스트를 벡터로 변환
    query_embedding = embeddings.embed_query(search_text)
    
    # 추가 필터링 조건 구성 - 사용자가 지정한 필터에 따라 쿼리 조건 생성
    filters = []

    # 장르 필터링 (예: Drama) - 특정 장르에 속한 영화만 검색하도록 필터 추가
    if genre:  
        filters.append("EXISTS { MATCH (node)-[:IN_GENRE]->(:Genre {name: $genre}) }")
    
    # 평점 필터링 (예: 7.5 이상) - 지정된 평점 이상의 영화만 검색하도록 필터 추가
    if min_rating:
        filters.append("node.rating >= $min_rating")
    
    # 필터 조건을 WHERE 절로 변환 - 필터가 있는 경우에만 WHERE 절 추가
    # 이 구조로 filter 조건을 계속 확장 가능 -> "끼워넣을문자열(AND)".join(리스트)
    where_clause = ""
    if filters:
        where_clause = "WHERE " + " AND ".join(filters)

    # Filter 조건 출력
    print(f"Filter 조건:\n{where_clause}\n\n")
    
    # 벡터 검색 쿼리 실행 - Neo4j의 벡터 인덱스를 활용한 의미적 검색 수행
    hybrid_search_query = f"""
    CALL db.index.vector.queryNodes(
        'movie_content_embeddings',  // 영화 콘텐츠 임베딩이 저장된 벡터 인덱스 이름
        100,                         // 초기 검색 결과 수 (더 많은 결과를 가져와서 필터링)
        $query_embedding             // 파라미터로 전달된 쿼리 임베딩 벡터
    ) YIELD node, score              // 검색된 노드와 유사도 점수 반환

    {where_clause}                   // 동적으로 생성된 필터링 조건 (장르, 평점 등)

    WITH node, score                 // 필터링된 노드와 점수를 다음 단계로 전달
    OPTIONAL MATCH (node)-[:IN_GENRE]->(g:Genre)  // 영화와 연결된 장르 노드 찾기 (백터 검색으로 찾아진 영화들은 현재 장르 관계가 없는 상태임으로...)

    RETURN node.title AS title,      // 영화 제목 반환
           node.released AS released,// 개봉일 반환
           node.rating AS rating,    // 평점 반환
           node.tagline AS tagline,  // 태그라인 반환
           node.overview AS overview, // 영화 개요 반환 
           collect(g.name) AS genres,// 모든 장르를 배열로 수집
           score AS similarity       // 유사도 점수 반환
           
    ORDER BY similarity DESC         // 유사도 점수 기준 내림차순 정렬
    LIMIT $top_k                     // 상위 k개 결과만 반환
    """
    
    # 쿼리 파라미터 설정 - 동적으로 필요한 파라미터만 포함
    params = {
        "query_embedding": query_embedding,
        "top_k": top_k
    }
    
    # 선택적 파라미터 추가 - 필터가 지정된 경우에만 해당 파라미터 추가
    if genre:
        params["genre"] = genre
    if min_rating:
        params["min_rating"] = min_rating
    
    # Neo4j 데이터베이스에 쿼리 실행 및 결과 반환
    results = graph.query(hybrid_search_query, params=params)
    
    return results

# 하이브리드 검색 테스트
hybrid_results = hybrid_movie_search(
    "가족간의 사랑과 신뢰 회복을 주제로 한 영화",
    genre="Drama",
    min_rating=7.0,
    top_k=5
)

# 검색 결과 출력 - 영화 제목, 유사도 점수, 평점, 장르 정보 표시
for result in hybrid_results:
    print(f"{result['title']} - 유사도: {result['similarity']:.4f}, " +
          f"평점: {result['rating']}, 장르: {', '.join(result['genres'])}")
    print(f"---- 태그라인: {result['tagline']}")
    print(f"---- 개요: {result['overview']}")
    print()

Filter 조건:
WHERE EXISTS { MATCH (node)-[:IN_GENRE]->(:Genre {name: $genre}) } AND node.rating >= $min_rating


The Best of Me - 유사도: 0.7193, 평점: 7.2, 장르: Drama, Romance
---- 태그라인: You never forget your first love.
---- 개요: A pair of former high school sweethearts reunite after many years when they return to visit their small hometown.

High Fidelity - 유사도: 0.7004, 평점: 7.0, 장르: Drama, Romance, Music, Comedy
---- 태그라인: A comedy about fear of commitment, hating your job, falling in love and other pop favorites.
---- 개요: When record store owner Rob Gordon gets dumped by his girlfriend, Laura, because he hasn't changed since they met, he revisits his top five breakups of all time in an attempt to figure out what went wrong. As Rob seeks out his former lovers to find out why they left, he keeps up his efforts to win Laura back.

Reign Over Me - 유사도: 0.6956, 평점: 7.1, 장르: Drama
---- 태그라인: Let in the unexpected.
---- 개요: A man who lost his family in the September 11 attack on New York City runs

In [10]:
# 하이브리드 검색 테스트 - 실제 검색 예시로 함수 실행
hybrid_results = hybrid_movie_search(
    "친구들의 우정을 그린 영화",
    min_rating=7.0,
    top_k=5
)

# 검색 결과 출력 - 영화 제목, 유사도 점수, 평점, 장르 정보 표시
for result in hybrid_results:
    print(f"{result['title']} - 유사도: {result['similarity']:.4f}, " +
          f"평점: {result['rating']}, 장르: {', '.join(result['genres'])}")
    print(f"---- 태그라인: {result['tagline']}")
    print(f"---- 개요: {result['overview']}")
    print()

Filter 조건:
WHERE node.rating >= $min_rating


Sleepers - 유사도: 0.6839, 평점: 7.3, 장르: Drama, Thriller, Crime
---- 태그라인: When friendship runs deeper than blood.
---- 개요: Two gangsters seek revenge on the state jail worker who during their stay at a youth prison sexually abused them. A sensational court hearing takes place to charge him for the crimes. A moving drama from director Barry Levinson.

Little White Lies - 유사도: 0.6819, 평점: 7.1, 장르: Drama, Comedy
---- 태그라인: The one thing friends can't escape is a few home truths.
---- 개요: Despite a traumatic event, a group of friends decide to go ahead with their annual beach vacation. Their relationships, convictions, sense of guilt and friendship are sorely tested. They are finally forced to own up to the little white lies they've been telling each other.

The Wood - 유사도: 0.6803, 평점: 7.1, 장르: Drama, Romance, Comedy
---- 태그라인: From best friends to best men.
---- 개요: In the panicky, uncertain hours before his wedding, a groom with prenuptial jitte